## CNN 모델 네트워크 구축

In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 데이터 불러오기

In [3]:
NUMPY_ROOT_PATH = 'C:/Users/PC00/Desktop/24하계 연수생/Handwriting_Classification/Numpy_data/'

In [4]:
X = np.load(NUMPY_ROOT_PATH+'20s_3_X.npy')
X = torch.tensor(X.reshape(-1, X.shape[3], X.shape[1], X.shape[2]), dtype=torch.float32)

In [5]:
y = np.load(NUMPY_ROOT_PATH+'20s_3_y.npy')
y = torch.tensor(y.reshape(-1), dtype=torch.long)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

In [8]:
print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

torch.Size([76144, 1, 76, 180]) torch.Size([9518, 1, 76, 180]) torch.Size([9518, 1, 76, 180])
torch.Size([76144]) torch.Size([9518]) torch.Size([9518])


### 모델 생성

In [9]:
from torch import nn
import torch.nn.functional as F
from torchsummary import summary

In [10]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()

        self.layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=3, stride=3),

            nn.Dropout2d(0.25),
            nn.Conv2d(32, 64, kernel_size=5, stride=1),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=3, stride=3),
            nn.Dropout2d(0.25)
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64*6*18, 128),
            nn.ReLU(),

            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.layer(x)
        out = out.view(-1, 64*6*18)
        out = self.fc_layer(out)

        return out

# GRID Search

In [11]:
learning_rate = [1e-3, 5e-4, 1e-4, 5e-5, 1e-5]
batch_size = [16, 32, 64, 128]
EPOCH = 200

In [12]:
def train_model(model, BATCH_SIZE, LEARNING_RATE, EPOCH):
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    top_valid_acc = 0

    for i in range(EPOCH):
        model.train()
        for j in range(0, len(X_train), BATCH_SIZE):
            X_batch = X_train[j:j+BATCH_SIZE].to(device)
            y_batch = y_train[j:j+BATCH_SIZE].to(device)

            optimizer.zero_grad()
            output = model(X_batch)

            loss = loss_func(output, y_batch)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred = model(X_valid.to(device))
            loss = loss_func(y_pred, y_valid.to(device))

            cur_valid_acc = (y_pred.argmax(1) == y_valid.to(device)).float().mean().item()

            if cur_valid_acc > top_valid_acc:
                top_valid_acc = cur_valid_acc

    return top_valid_acc

In [13]:
result = []

for lr in learning_rate:
    for bs in batch_size:
        model = LeNet().to(device)
        result.append({
            'learning_rate': lr,
            'batch_size': bs,
            'top_valid_acc': train_model(model, bs, lr, EPOCH)
        })
        print(f'learning rate: {lr}, batch size: {bs}, top valid acc: {result[-1]["top_valid_acc"]}')
        del model

result_df = pd.DataFrame(result)
result_df

learning rate: 0.001, batch size: 16, top valid acc: 0.7360789775848389
learning rate: 0.001, batch size: 32, top valid acc: 0.7430132031440735
learning rate: 0.001, batch size: 64, top valid acc: 0.7517335414886475
learning rate: 0.001, batch size: 128, top valid acc: 0.7472158074378967
learning rate: 0.0005, batch size: 16, top valid acc: 0.7435385584831238
learning rate: 0.0005, batch size: 32, top valid acc: 0.748686671257019
learning rate: 0.0005, batch size: 64, top valid acc: 0.7502626180648804
learning rate: 0.0005, batch size: 128, top valid acc: 0.7505778074264526
learning rate: 0.0001, batch size: 16, top valid acc: 0.7518386244773865
learning rate: 0.0001, batch size: 32, top valid acc: 0.7461651563644409
learning rate: 0.0001, batch size: 64, top valid acc: 0.7469005584716797
learning rate: 0.0001, batch size: 128, top valid acc: 0.7505778074264526
learning rate: 5e-05, batch size: 16, top valid acc: 0.7501575946807861
learning rate: 5e-05, batch size: 32, top valid acc: 0

,learning_rate,batch_size,top_valid_acc
0,0.00100,16,0.736079
1,0.00100,32,0.743013
2,0.00100,64,0.751734
3,0.00100,128,0.747216
4,0.00050,16,0.743539
5,0.00050,32,0.748687
6,0.00050,64,0.750263
7,0.00050,128,0.750578
8,0.00010,16,0.751839
9,0.00010,32,0.746165


In [14]:
result_df.to_csv('./LeNet_result.csv', encoding = 'utf-8', index=False)